In [3]:
# from transformers import AutoTokenizer, EsmForSequenceClassification
import torch

In [4]:
# Can be used
# model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t30_150M_UR50D")
# model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t36_3B_UR50D")
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t33_650M_UR50D")

Using cache found in /home/nikola_dev/.cache/torch/hub/facebookresearch_esm_main


In [5]:
# import esm

In [6]:
# model, alphabet = esm.pretrained.esm2_t30_150M_UR50D()

In [7]:
# model has a base class nn.Module
# print(esm.model.esm2.ESM2.__bases__)
model

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [8]:
from torch.nn import Identity
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.contact_head = Identity()
model.emb_layer_norm_after = Identity()
model.lm_head = Identity()
model

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): Identity()
  (emb_layer_norm_after): Identity()
  (lm_head): Identity()
)

In [9]:
model = model.to(device)
model.eval()

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): Identity()
  (emb_layer_norm_after): Identity()
  (lm_head): Identity()
)

In [10]:
import pandas as pd
df = pd.read_csv('../transformer/pfam_1000.train.csv')

In [11]:
df['original'] = df['original'].str.slice(0, 1010)

In [15]:
batch_converter = alphabet.get_batch_converter()
batch_labels, batch_strs, batch_tokens = batch_converter(df[['label', 'original']].values)

In [18]:
# del batch_tokens
# import gc         # garbage collect library
# gc.collect()
# torch.cuda.empty_cache()
batch_tokens.shape

torch.Size([70, 1012])

In [16]:
%%time
batch_tokens = batch_tokens[:70]
batch_tokens = batch_tokens.to(device)

CPU times: user 323 µs, sys: 0 ns, total: 323 µs
Wall time: 227 µs


In [17]:
%%time
with torch.no_grad():
    res = model(batch_tokens)

CPU times: user 940 ms, sys: 2.04 s, total: 2.98 s
Wall time: 3.12 s


{'logits': tensor([[[  2.5846,   4.5874, -30.1158,  ...,  40.4336, -13.2980, -26.7201],
          [-29.8954, -35.8522, -13.1067,  ..., -12.0015, -19.4289, -13.1325],
          [-16.0272, -11.9577,  -9.8021,  ...,  -2.5943, -17.9272,  19.2647],
          ...,
          [-10.0251,   3.5973, -17.1696,  ...,  30.7033, -34.2063, -32.8483],
          [  1.0403,  -2.1441, -16.3068,  ...,  26.6433, -37.7713, -36.5021],
          [  4.2510,  -0.2660,  -9.6696,  ...,  24.8633, -33.6851, -42.5567]],
 
         [[  3.3450,   4.8753, -29.8508,  ...,  40.6506, -12.4902, -26.9086],
          [-26.5051, -35.8559, -12.6460,  ..., -12.7459, -17.9250, -13.8510],
          [-12.6986, -13.6579,  -9.5891,  ...,  -1.0904, -15.8776,  16.5130],
          ...,
          [ -4.8533, -21.7571,  -4.0157,  ...,  22.8195, -32.4116,  -1.9318],
          [ -2.5501,  -4.6750,  -6.2363,  ...,  18.1955, -34.2790,   2.3410],
          [ -5.2621,   3.3198, -15.4875,  ...,  23.8405, -35.9860, -32.6736]]],
        device='cud

In [120]:
mod = esm.model.esm2.ESM2()

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [134]:
type(model)

esm.model.esm2.ESM2